# Manejo de variables

## Recomendaciones para mejorar la eficiencia de codigo

Julia es un lenguaje de programacion en el cual los tipos de variables son fundamentales para hacer los procesos de manera eficiente, hablamos del despacho múltiple, que no abrodaremos mucho, pero en esencia Julia identifica el tipo de variable que se utiliza si no la definimos explicitamente. <br>Sin embargo, la idea es facilitar el procesamiento de nuestro codigo por lo cual tomando en cuenta lo anterior hay aspectos que podemos mejorar a la hora de programar.

### Evitar variables de tipo abstracto

Para simplificar este concepto, podemos pensar en lo siguiente:
-  Un *tipo abstracto* seria como *Real* (conjunto de los numeros reales).
-  Un *tipo concreto* podria ser Int64, Float64, etc. 

In [1]:
#Define una funcion que muestra las ramas de un tipo de variable.
function _show_subtype_tree(mytype,printlevel)
    allsubtypes = subtypes(mytype)
    for cursubtype in allsubtypes
        print("\t"^printlevel)
        println("|___",cursubtype)
        printlevel += 1
        _show_subtype_tree(cursubtype,printlevel)
        printlevel -= 1
    end
end
function show_type_tree(T)
    println(T)
    _show_subtype_tree(T,0)
end
show_type_tree(Number) #mostramos los subtipos de Number

Number
|___Complex
|___Real
	|___AbstractFloat
		|___BigFloat
		|___Float16
		|___Float32
		|___Float64
	|___AbstractIrrational
		|___Irrational
	|___Integer
		|___Bool
		|___Signed
			|___BigInt
			|___Int128
			|___Int16
			|___Int32
			|___Int64
			|___Int8
		|___Unsigned
			|___UInt128
			|___UInt16
			|___UInt32
			|___UInt64
			|___UInt8
	|___Rational


In [2]:
#Podemos asegurarnos si el tipo de variable es concreto o abstracto
@show isconcretetype(Float64);
@show isconcretetype(Real);

isconcretetype(Float64) = true
isconcretetype(Real) = false


In [3]:
using BenchmarkTools

In [4]:
#Juego de lanzamiento de moneda.
#Contabiliza el numero de veces que cae en una u otra cara, es decir cuantas veces "ganamos".

function record_games_won(ngames)
    games_won = [] #definimos solo el arreglo y julia interpreta el tipo
    for i = 1:ngames
        r = rand()
        if r >= 0.5
            push!(games_won,i)
        end
    end
    return games_won
end
function record_games_won_v2(ngames)
    games_won = Int64[] #definimos un arreglo de tipo entero
    for i = 1:ngames
        r = rand()
        if r >= 0.5
            push!(games_won,i)
        end
    end
    return games_won
end
record_games_won(2);
record_games_won_v2(2);

ntrials = 1000
@btime record_games_won(ntrials);
@btime record_games_won_v2(ntrials);

  10.300 μs (197 allocations: 10.53 KiB)
  8.500 μs (5 allocations: 7.55 KiB)


Podemos observar los tiempos y uso de asignacion de memoria y como mejora cuando definimos un tipo de variable.

### Evitar entorno global

Cuando programamos usualmente definimos nuestro codigo en funciones, ya sea por rendimiento o para una mejor ejecución o facilitar la comprehensión y usabilidad. Asi que todo lo que definimos dentro de dichas funciones se convierte en nuestro entorno local, y dichas funciones las llamamos al entorno global, que en este caso pueden ser el REPL o jupiter notebook por ejemplo.

In [24]:
allgames = rand(ntrials)
function record_games_won_global()
    games_won = Int64[]
    for (current_index,current_game) in enumerate(allgames)#iteramos en allgames con la funcion enumerate() en el ciclo for
        if current_game >= 0.5                     #que produce un contador asociado al iterador actual (index,array_input)
            push!(games_won,current_index) #guardamos en el arreglo definido(games_won en indice del evento)
        end
    end
    return games_won  #regresamos el arreglo que contiene los indices de los juegos "ganados"
end

function record_games_won_local(ntrials)
    allgames = rand(ntrials)#definimos de manera local el arreglo de juegos ganados
    games_won = Int64[]
    for (curi,curgame) in enumerate(allgames)
        if curgame >= 0.5
            push!(games_won,curi)
        end
    end
    return games_won
end

record_games_won_global();
record_games_won_local(ntrials);

@btime record_games_won_global();
@btime record_games_won_local(ntrials);

  100.200 μs (4495 allocations: 140.20 KiB)
  7.500 μs (6 allocations: 15.48 KiB)


Podemos observar que al definir dentro de la función el arreglo y solo pasar como argumento el numero de muestras(juegos)  la ejecución utiliza menos memoria y tiempo pues no utiliza necesita acceder a una dirección en memoria y solo se genera durante la ejecución.

**Nota:** Si es necesario crear una variable global, definirla como *const* servirá, esto no implica que no cambie de valo sino que no se modificara el tipo de variable durante la ejecución.

In [25]:
const myglobalint = 1
myglobalint = 2
@show myglobalint

myglobalint = 2


2

In [26]:
myglobalint = 1.5

LoadError: invalid redefinition of constant myglobalint

### Preasignar memoria

En este caso debemos tomar en cuenta nuestra funcion y que nos devuelve, si es un arreglo u otro tipo de valor complejo deberiamos asignar memoria.
A veces podemos evitar la necesidad de asignar memoria en cada llamada de función preasignando la salida.

In [37]:
function record_games_won_v2(ngames)
    games_won = Int64[]
    for i = 1:ngames
        r = rand()
        if r >= 0.5
            push!(games_won,i)
        end
    end
    return games_won
end


function record_games_won_preallocate(ntrials)
    allgames = rand(ntrials)
    games_won = Vector{Int64}(undef,ntrials)#creamos un vector sin inicializar de tipo entero para los registros de juego
    game_index = 1
    for (current_index,current_game) in enumerate(allgames)
        if current_game >= 0.5
            games_won[game_index] = current_index
            game_index += 1
        end
    end
    return games_won[1:game_index-1]
end


record_games_won_preallocate(ntrials);

@btime record_games_won_v2(ntrials);
@btime record_games_won_preallocate(ntrials);

  7.967 μs (5 allocations: 7.55 KiB)
  4.650 μs (3 allocations: 19.73 KiB)


Se redujo el tiempo y la asignación de memoria debido a que hay un espacio preasignado del tamaño de la muestra inicial, por lo que ya no se reserva mas memoria de la necesaria.

### Uso de operaciones vectorizadas fusionadas (difusión/broadcast)

Dentro de las funciones de Julia para realizar operaciones matemáticas y analizar datos,una función escalar se aplica sobre variables escalares y tambien sobre vectores, sin embargo, las funciones solamente haran un elemento a la vez un ejemplo seria sin(x), exp(x), abs(x), etcétera, a no ser que se realice un ciclo para iterar.<br>Julia posee una sintaxis especial para el punto, el cual convierte una función escalar en una función *vectorizada* y un operador escalar en un operador *vectorizado*, esto además incluye que las llamadas de punto para *vectorizar* que estan anidadas se fusionan, es decir, se combinan sintacticamente en un solo bucle sin necesidad de crear matrices temporales por lo que mejoramos la eficiencia en recursos de memoria y tiempo.


In [40]:
function record_games_won_preallocate_fused(ntrials)
    allgames = rand(ntrials)
    games_won = findall(allgames.>=0.5)
    return games_won
end

record_games_won_preallocate_fused(ntrials);
@btime record_games_won_v2(ntrials);#Version del ejemplo anterior con preasignación de memoria
@btime record_games_won_preallocate_fused(ntrials);

  7.933 μs (5 allocations: 7.55 KiB)
  1.390 μs (5 allocations: 16.25 KiB)


Con esto, de primera instancia logramos reducir el código visualmente comparado con la ultima versión de la funcion(*record_games_won_preallocate()*) y además al no realizar una iteración de forma manual mediante un *ciclo for* mejoramos el tiempo.

In [45]:
#Otro ejemplo de una función escalar aplicando el operador punto. 
f(x) = 3x.^2 + 4x + 7x.^3; #Sobre otros operadores(^)
g(x) = @. 3x^2 + 4x + 7x^3 #tambien se puede definir con una macro que equivaldra a 3 .* x.^2 .+ 4 .* x .+ 7 .* x.^3
x = rand(1000)
@btime f(x);
@btime f.(x);#Sobre la función misma
@btime g(x);
@btime g.(x);

  2.725 μs (6 allocations: 47.62 KiB)
  751.818 ns (4 allocations: 8.00 KiB)
  531.694 ns (1 allocation: 7.94 KiB)
  727.473 ns (4 allocations: 8.00 KiB)


Podemos observar los tiempos y notar que ultimas tres definiciones son mas rápidas sin embargo en las ultimas dos notamos que no siempre vectorizar sobre una función ya vectorizada es lo mejor puesto que se crean mas ciclos de los necesarios, lo ideal seria esparcir los puntos a lo largo de la función, definir un macro o aplicar sobre la función completa, segun sea el caso

### No necesitas " *vectorizar* "

In [51]:
function find_hypotenuse_vectorized(b,h)
    return sqrt.(b.^2+h.^2)
end

function find_hypotenuse_forloop(b,h)
    accum_vec = similar(b)#Preasignamos en memoria un arreglo de tipo y tamaño como el de b
    for i = 1:length(b)
        accum_vec[i] = sqrt(b[i]^2+h[i]^2) #iteramos sobre el arreglo b y h
    end
    return accum_vec
end

b = rand(ntrials)
h = rand(ntrials)
@btime find_hypotenuse_vectorized($b,$h);
@btime find_hypotenuse_vectorized.($b,$h);
@btime find_hypotenuse_forloop($b,$h);

  2.788 μs (4 allocations: 31.75 KiB)
  800.000 ns (1 allocation: 7.94 KiB)
  1.480 μs (1 allocation: 7.94 KiB)


### Reutilizar memoria

Este ejemplo es similar a la preasignación de memoria sin embargo el objetivo aqui es liberar la memoria al terminar la ejecución de a función

In [73]:
function find_sum_of_sqrt_vectors(nvectors)
    sumvector = Vector{Float64}(undef,nvectors)
    for i = 1:nvectors
        # v = sqrt.(1:i)
        sumvector[i] = sum(sqrt.(1:i))
    end
    return sumvector
end

function find_sum_of_sqrt_vectors_reusemem(nvectors)
    sumvector = Vector{Float64}(undef,nvectors)
    v = Vector{Float64}(undef,nvectors)
    for i in 1:nvectors
        v[1:i] .= sqrt.(1:i)
        sumvector[i] = sum(v)
        v .= 0
    end
    return sumvector
end

@btime find_sum_of_sqrt_vectors($ntrials);
@btime find_sum_of_sqrt_vectors_reusemem($ntrials);

  596.100 μs (1001 allocations: 3.96 MiB)
  461.200 μs (2 allocations: 15.88 KiB)


### Usar @view cuando no necesites una copia de la información

In [112]:
using SparseArrays
using LinearAlgebra
A = sprand(500,500,0.1)
function set_sum(A,rowids,colids)
    s = sum(A[rowids,colids])
end
function set_sum_view(A,rowids,colids)
    s = sum(view(A,rowids,colids))
end

using Random
@btime set_sum(A,randperm(10), randperm(10));
@btime set_sum_view(A,randperm(10), randperm(10));

  1.844 μs (17 allocations: 5.47 KiB)
  1.290 μs (3 allocations: 304 bytes)


### Accede a las matrices por columna primero

Las matrices se ordenan con un indice de orden mayor por columna, esto es que una matriz de orden Mat[m,n] se puede ver tambien como Mat[m*n] donde el indice recorre el arreglo por columnas, es decir, una matriz de tipo 
\begin{pmatrix}
1 & 2 \\
a & b
\end{pmatrix} se ordenaria como un arreglo de la forma \begin{pmatrix}
1 & b & 2 & b
\end{pmatrix}

In [121]:
m = ntrials
n = 10000
A = rand(m,n)

function matrix_sum_rows(A)
    m,n = size(A)
    mysum = 0
    for i = 1:m # fix a row
        for j = 1:n # loop over cols
            mysum += A[i,j]
        end
    end
    return mysum
end

function matrix_sum_cols(A)
    m,n = size(A)
    mysum = 0
    for j = 1:n # fix a column
        for i = 1:m # loop over rows
            mysum += A[i,j]
        end
    end
    return mysum
end

function matrix_sum_index(A)
    m,n = size(A)
    mysum = 0
    for i = 1:m*n
        mysum += A[i]
    end
    return mysum
end
@btime matrix_sum_rows($A);
@btime matrix_sum_cols($A);
@btime matrix_sum_index($A);

  41.008 ms (0 allocations: 0 bytes)
  37.221 ms (0 allocations: 0 bytes)
  37.841 ms (0 allocations: 0 bytes)


Por lo tanto en este ejemplo podemos ver que el tiempo para recorrer el arreglo es menor si recorremos la matriz primero por columnas y luego por filas o en este caso por columnas y como arreglo de orden mayor que por filas, el tiempo es muy similar tanto en *matrix_sum_cols()* como en *matrix_sum_index()*

In [116]:
A = rand(m,n);

In [117]:
m,n = size(A)

(1000, 10000)

In [118]:
m

1000